In [ ]:
from sklearn.preprocessing import MinMaxScaler,QuantileTransformer
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
from tqdm import tqdm
from ann import ML,seed_everything
net_list=[878,603]#same net
net_i=0

def get_dataset(main_path,error):
    print('dataset construction')
    dataset=pd.DataFrame([])
    for i in tqdm(range(net_list[net_i])):
        set=pd.read_csv(main_path+'dataset\\'+error+'\\sample_'+error+'_pipe_'+str(i)+'_residual_feature_labeled.csv',header=0)
        dataset=pd.concat([dataset,set],axis=0)
    return dataset.reset_index(drop=True)

print('---------------------')
seed_everything(42)
path='D:\\0-Data\\9-stcleak3\\'

error_list=['0100']
error=error_list[0]

dataset=get_dataset(path,error)
dataset_x=np.array(dataset.iloc[:,10:])
dataset_y=np.array(dataset.loc[:,'Label']).reshape(-1,1).astype(int)#Code Label
print('dataset size:',dataset_x.shape)
print('label size:',len(np.unique(dataset_y)))

In [ ]:
splites_num=5
skf = StratifiedKFold(n_splits=splites_num)
for i, (train_index, test_index) in enumerate(skf.split(dataset_x, dataset_y)):
    x_train,x_test=dataset_x[train_index,:],dataset_x[test_index,:]
    y_train,y_test=dataset_y[train_index,:],dataset_y[test_index,:]
    test_set=dataset.iloc[test_index,:10].reset_index(drop=True)

    sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
    x_train=sel.fit_transform(x_train)
    x_test=sel.transform(x_test)
    print('train set size sel:',x_train.shape)

    pca = PCA(n_components='mle')
    x_train=pca.fit_transform(x_train)
    x_test = pca.transform(x_test)
    print('train set size pca:',x_train.shape)

    qt=QuantileTransformer()
    x_train = qt.fit_transform(x_train)
    x_test = qt.transform(x_test)

    hs=500
    name1='hid'+str(hs)+'_stc'+str(net_list[net_i])+'_l'+str(len(np.unique(y_train)))+'_cv'+str(i)+'of5'
    name2=name1
    # name2='hid'+str(hs)+'_stc'+str(net_list[net_i])+'_l'+str(len(np.unique(y_train)))+'_cv0of5'+'_'+error_test

    params={
    'indim':x_train.shape[1],
    'hiddim': hs,
    'outdim':len(np.unique(y_train)),

    'epochs':250,
    'learning_rate':0.01,
    'weight_decay':0.01,
    'batch_size':int(len(x_train)*0.1)+1,

    'train_name':name1,
    'test_name':name2,
    'test_set':test_set,
    'x_train':x_train,
    'y_train':y_train,
    'x_test':x_test,
    'y_test':y_test,
    }

    ml=ML(**params)
    ml.set_path(path)
    ml.train(draw=False)
    ml.load()#load best model
    ml.output('all')